In [15]:
!pip install langgraph

In [16]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

In [17]:
from langchain_ollama import ChatOllama
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

@tool
def search_tool(query: str):
    """search the web for info using DuckDuckGo."""
    search_tool = DuckDuckGoSearchRun()
    result = search_tool.invoke(query)
    return result

tools = [search_tool]
tool_node = ToolNode(tools)


# setup llm node
model = ChatOllama(model="llama3.2:latest").bind_tools(tools)

def call_model(state: AgentState):
    response = model.invoke(state["messages"])
    return response
    #return a list


In [18]:
def should_continue(state: AgentState):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return "__end__"

In [19]:
from langgraph.graph import StateGraph, START

workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# add edge from start to agent
workflow.add_edge(START, "agent")

# add conditional edge from agent to tools or end
workflow.add_conditional_edges("agent", should_continue, )

# add edge from tools to agent
workflow.add_edge("tools", "agent")

# compile the graph
app = workflow.compile()

In [21]:
final_state = app.invoke({"messages": [("user", "Who is the president of the USA?") ]})
print(final_state["messages"][-1].content)

AttributeError: 'HumanMessage' object has no attribute 'tool_calls'